In [2]:
import sys

In [3]:
sys.path.insert(0, '/home/joe2/Documents/LambdaSchool/labs_ir/repos/Data-Science/data')

In [4]:
from fin_data import DailyTimeSeries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import scipy as sp
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM
%matplotlib inline

Using TensorFlow backend.


In [12]:
from keras.preprocessing.sequence import TimeseriesGenerator


In [5]:
apple = DailyTimeSeries('AAPL')
dfaapl = apple.initiate()
apple.list_available_fundamentals()

################################################################### 
 Ticker:  AAPL 
 Last Refreshed:  2019-09-03 
 Data Retrieved:  Daily Prices (open, high, low, close) and Volumes 
 ###################################################################


['operatingrevenue',
 'totalrevenue',
 'operatingcostofrevenue',
 'totalcostofrevenue',
 'totalgrossprofit',
 'sgaexpense',
 'rdexpense',
 'totaloperatingexpenses',
 'totaloperatingincome',
 'otherincome',
 'totalotherincome',
 'totalpretaxincome',
 'incometaxexpense',
 'netincomecontinuing',
 'netincome',
 'netincometocommon',
 'weightedavebasicsharesos',
 'basiceps',
 'weightedavedilutedsharesos',
 'dilutedeps',
 'weightedavebasicdilutedsharesos',
 'basicdilutedeps',
 'cashdividendspershare',
 'date',
 'fiscal_year',
 'quarter']

In [6]:
funs = ['totalrevenue',  'totalcostofrevenue', 'totalgrossprofit', 'totalpretaxincome', 'weightedavebasicdilutedsharesos', 'cashdividendspershare', 'quarter']
techs = ['SMA', 'WMA', 'STOCH', 'ROC', 'AROON']
macros = ['housing_index', 'confidence_index', 'trade_index', 'longterm_rates', 'shortterm_rates']

In [7]:
dfaapl = apple.add_fundamentals(dfaapl, funs)
dfaapl = apple.add_technicals(techs, dfaapl)
dfaapl = apple.add_macro(dfaapl, macros)

################################################################### 
 Ticker:  AAPL 
 Fundamentals Retrieved:  ['AAPL open' 'AAPL high' 'AAPL low' 'AAPL close' 'AAPL volume'
 'AAPL_totalrevenue' 'AAPL_totalcostofrevenue' 'AAPL_totalgrossprofit'
 'AAPL_totalpretaxincome' 'AAPL_weightedavebasicdilutedsharesos'
 'AAPL_cashdividendspershare' 'AAPL_quarter'] 
 ###################################################################
################################################################### 
 Ticker:  AAPL 
 Retrieved Data Start Date:  2009-07-22 
 Retrieved Data End Date:  2019-07-31 
 Data Retrieved:  ['AAPL_totalrevenue', 'AAPL_totalcostofrevenue', 'AAPL_totalgrossprofit', 'AAPL_totalpretaxincome', 'AAPL_weightedavebasicdilutedsharesos', 'AAPL_cashdividendspershare', 'AAPL_quarter'] 
 ###################################################################
################################################################### 
 Ticker:  AAPL 
 Last Refreshed:  Simple Moving Average (SMA) 
 Da

/home/joe2/Documents/LambdaSchool/labs_ir/repos/Data-Science/data/fin_data.py:293: UserWarning: The latest value available for Housing Index is from January 2019.
  warnings.warn("The latest value available for Housing Index is from January 2019.")


################################################################### 
 Index: Yale Investor Behavior Project Added 
 ###################################################################
################################################################### 
 Trade Weighted U.S. Dollar Index: Broad Added 
 ###################################################################
################################################################### 
 US Treasury Bond Long-Term Rates Added 
 ###################################################################
################################################################### 
 US Treasury Bond Short-Term Rates Added 
 ###################################################################


/home/joe2/Documents/LambdaSchool/labs_ir/repos/Data-Science/data/fin_data.py:334: UserWarning: Contains Null Values
  warnings.warn("Contains Null Values")


In [8]:
dfaapl.head()

,AAPL open,AAPL high,AAPL low,AAPL close,AAPL volume,AAPL_totalrevenue,AAPL_totalcostofrevenue,AAPL_totalgrossprofit,AAPL_totalpretaxincome,AAPL_weightedavebasicdilutedsharesos,...,4_Wk_DR,4_Wk_CE,8_Wk_DR,8_Wk_CE,13_Wk_DR,13_Wk_CE,26_Wk_DR,26_Wk_CE,52_Wk_DR,52_Wk_CE
date,,,,,,,,,,,,,,,,,,,,,
2019-01-31,166.11,169.00,164.56,166.44,40739600.0,8.829300e+10,5.438100e+10,3.391200e+10,2.703000e+10,5.118600e+09,...,2.38,2.42,2.39,2.43,2.36,2.41,2.40,2.46,2.47,2.55
2019-01-30,163.25,166.15,160.23,165.25,61109800.0,8.829300e+10,5.438100e+10,3.391200e+10,2.703000e+10,5.118600e+09,...,2.36,2.40,2.35,2.39,2.37,2.42,2.44,2.50,2.49,2.57
2019-01-29,156.25,158.13,154.11,154.68,41587200.0,4.540800e+10,2.792000e+10,1.748800e+10,1.130800e+10,5.188700e+09,...,2.35,2.39,2.37,2.41,2.37,2.42,2.45,2.52,2.52,2.60
2019-01-28,155.79,156.33,153.66,156.30,26192100.0,4.540800e+10,2.792000e+10,1.748800e+10,1.130800e+10,5.188700e+09,...,2.35,2.39,2.37,2.41,2.37,2.42,2.45,2.52,2.52,2.60
2019-01-25,155.48,158.13,154.32,157.76,33535500.0,4.540800e+10,2.792000e+10,1.748800e+10,1.130800e+10,5.188700e+09,...,2.32,2.36,2.37,2.41,2.34,2.39,2.45,2.51,2.52,2.60


In [9]:
dfaapl.isnull().sum()

AAPL open                                  0
AAPL high                                  0
AAPL low                                   0
AAPL close                                 0
AAPL volume                                0
AAPL_totalrevenue                       1901
AAPL_totalcostofrevenue                 1901
AAPL_totalgrossprofit                   1901
AAPL_totalpretaxincome                  1901
AAPL_weightedavebasicdilutedsharesos    1901
AAPL_cashdividendspershare              1901
AAPL_quarter                            1901
AAPL_SMA                                   0
AAPL_WMA                                   0
AAPL_SlowD                                 0
AAPL_SlowK                                 0
AAPL_ROC                                   0
AAPL_Aroon Up                              0
AAPL_Aroon Down                            0
housing_index                              0
conf_index                                 0
conf_index_SE                              0
trade_valu

In [10]:
dfaapl = dfaapl.drop(labels=['8_Wk_DR', '8_Wk_CE'], axis=1)

In [11]:
dfaapl = dfaapl.dropna(axis=0)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
generator = TimeseriesGenerator()